In [1]:
%pylab inline
import bokeh
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Range1d, LabelSet, Label
output_notebook()

from glob import glob
import gensim
from nltk.corpus import stopwords

from sklearn.decomposition import PCA

import torch
from torch import nn
from torch.nn import functional as F
from torch import optim

from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy
from time import time
from pathlib import Path

Populating the interactive namespace from numpy and matplotlib


Loading BokehJS ...

In [2]:
corpus, target = [], []

stopw = stopwords.words("english")
for file in glob("dataset/*labelled.txt"):
    texts = open(file).read()
    i = 0
    for line in texts.splitlines():
        
        try:
            target.append(int(line.split("\t")[-1]))
            
            sc = gensim.utils.simple_preprocess(line) # sentence candidate
            sc = [i for i in sc if i not in stopw]
            corpus.append(sc)
        except:
            print(file, i)
        i += 1

assert len(corpus) == len(target), f"corpus: {len(corpus)} | target: {len(target)}"

dataset/imdb_labelled.txt 178
dataset/imdb_labelled.txt 968


In [3]:
max_sentence = 0
for i in corpus:
    len_sentence = len(i)
    if len_sentence > max_sentence:
        max_sentence = len_sentence
print("max_sentence: ", max_sentence)

max_sentence:  44


In [4]:
m = max_sentence
n = 10

In [5]:
skip_gram = gensim.models.Word2Vec(corpus, size=n, sg=1, iter=2000, min_count=1)

In [6]:
pca = PCA(n_components=2)
XY = pca.fit_transform(skip_gram[skip_gram.wv.vocab])

/home/lincoln/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [7]:
source = ColumnDataSource(data=dict(x=XY[:, 0], y=XY[:, 1], labels=list(skip_gram.wv.vocab.keys())))
p = figure(plot_width=600, plot_height=600)
p.scatter(x='x', y='y', size=8, source=source)
labels = LabelSet(x='x', y='y', text='labels', level='glyph',
              x_offset=5, y_offset=5, source=source, render_mode='canvas')

p.add_layout(labels)
show(p)

In [8]:
del XY, source, p, labels, pca

In [9]:
def make_matrix(phrase, model_vec, m, n, stop_words):
    if type(phrase) is str:
        phrase = gensim.utils.simple_preprocess(phrase)
        phrase = [i for i in phrase if i not in stop_words]
    
    out = np.zeros(m*n)
    for i, label in zip(range(0, len(phrase)*n, n), phrase):
        if label in model_vec.wv.vocab.keys():
            out[i:i+n] = model_vec[label]

    return out.reshape(1, -1)

In [10]:
make_matrix(corpus[442], skip_gram,m, n, stopw).shape

/home/lincoln/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


(1, 440)

In [11]:
class Data(Dataset):
    def __init__(self, data, target, model, m, n, stop_words):
        self.n = n
        self.m = m
        self.data = torch.FloatTensor(
            [make_matrix(i, model, m, n, stop_words) for i in data]
        )
        self.target =  torch.LongTensor([int(i) for i in target])
        self._len = len(target)
        
    def __getitem__(self, x):
        return self.data[x], self.target[x]
    
    def __len__(self):
        return self._len

In [12]:
data = Data(corpus, target, skip_gram, m, n, stopw)

/home/lincoln/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


In [13]:
data[4][0].shape

torch.Size([1, 440])

In [14]:
train_data, test_data, train_target, test_target = train_test_split(corpus, target, test_size=0.25)

data_train = Data(train_data, train_target, skip_gram, m, n, stopw)
data_test = Data(test_data, test_target, skip_gram, m, n, stopw)

/home/lincoln/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


In [15]:
print(f"\
lengh train: {data_train._len}\n\
lengh test: {data_test._len}")

lengh train: 2250
lengh test: 750


In [16]:
class CNN(nn.Module):
    def __init__(self, lin_in, lin_out):
        super(CNN, self).__init__()
        
        self.conv = nn.Sequential(
            nn.Conv1d(1, 16, 5, stride=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=5)
        )
        
        self.dropout = nn.Dropout(p=0.2)
        
        self.linear0 = nn.Linear(lin_in, int(lin_in*2))
        self.linear1 = nn.Linear(int(lin_in*2), int(lin_in/2))
        self.linear2 = nn.Linear(int(lin_in/2), lin_out)
        
    def forward(self, x):
        x = self.conv(x)
        
        x = x.view(x.shape[0], -1)
        x = self.linear0(x)
        x = self.dropout(x)
        x = self.linear1(x)
        x = torch.sigmoid(self.linear2(x))
        return x
        

In [17]:
cnn = CNN(129, 2)
t = next(iter(DataLoader(data_train, batch_size=3)))
print(cnn(t[0]))
del t

tensor([[0.5434, 0.4983],
        [0.5332, 0.4744],
        [0.5549, 0.5384]], grad_fn=<SigmoidBackward>)


In [18]:
train_loader = DataLoader(data_train, batch_size=int(data_train._len/3), shuffle=True)
test_loader = DataLoader(data_test, batch_size=data_test._len)

In [19]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn.parameters(), lr=0.001)

trainer = create_supervised_trainer(
    cnn, optimizer, criterion
)

evaluator = create_supervised_evaluator(
    cnn,
    metrics={
        "accuracy": Accuracy()
    },
    non_blocking=True
)

In [20]:
@trainer.on(Events.STARTED)
def start_prepare(trainer):
    trainer._loss = []
    trainer._accuracy_train = []
    trainer._accuracy_test = []
    trainer._init_time = time()
    trainer._csv_path = "results.csv"
    if not Path(trainer._csv_path):
        with open(trainer._csv_path, "x") as f:
            f.write("epoch, loss, accuracy_train, accuracy_test, model\n")
            f.close()

@trainer.on(Events.EPOCH_COMPLETED)
def registry_progress(trainer):
    evaluator.run(train_loader)
    loss = trainer.state.output
    trainer._loss.append(
        loss
    )
    accuracy_train = evaluator.state.metrics["accuracy"]
    trainer._accuracy_train.append(
        accuracy_train
    )
    
    evaluator.run(test_loader)
    accuracy_test = evaluator.state.metrics["accuracy"]
    trainer._accuracy_test.append(
        accuracy_test
    )

    with open(trainer._csv_path, "a") as f:
        f.write(f"{trainer.state.epoch}, {loss}, {accuracy_train}, {accuracy_test}, cnn_1d\n")
        f.close()

@trainer.on(Events.EPOCH_COMPLETED)
def progress_log(trainer):
    if trainer.state.epoch % 10 == 0:
        evaluator.run(train_loader)
        metrics = evaluator.state.metrics
        print(f"epoch: {trainer.state.epoch:<2} | " + \
              f"accuracy: {metrics['accuracy']:.2f} | " + \
              f"loss: {trainer.state.output:.2f}")
              
@trainer.on(Events.COMPLETED)
def summary(trainer):
    print(f"\n\
    ---\n\
    epochs: {trainer.state.epoch}\n\
    total time: {time() - trainer._init_time:.0f} seconds\n\n\
    \t\t loss \t train \t test \n\
    init\t {trainer._loss[0]:.3f} \t {trainer._accuracy_train[0]:.3f} \t {trainer._accuracy_test[0]:.3f}\n\
    end\t {trainer._loss[-1]:.3f} \t {trainer._accuracy_train[-1]:.3f} \t {trainer._accuracy_test[-1]:.3f}\n\
    ---")
    torch.save(cnn, "saved_models/cnn_1d.pth")


In [21]:
state = trainer.run(train_loader, max_epochs=150)

epoch: 10 | accuracy: 0.59 | loss: 0.67
epoch: 20 | accuracy: 0.63 | loss: 0.65
epoch: 30 | accuracy: 0.65 | loss: 0.64
epoch: 40 | accuracy: 0.66 | loss: 0.62
epoch: 50 | accuracy: 0.66 | loss: 0.62
epoch: 60 | accuracy: 0.67 | loss: 0.62
epoch: 70 | accuracy: 0.68 | loss: 0.61
epoch: 80 | accuracy: 0.68 | loss: 0.60
epoch: 90 | accuracy: 0.68 | loss: 0.61
epoch: 100 | accuracy: 0.68 | loss: 0.62
epoch: 110 | accuracy: 0.68 | loss: 0.62
epoch: 120 | accuracy: 0.68 | loss: 0.60
epoch: 130 | accuracy: 0.68 | loss: 0.58
epoch: 140 | accuracy: 0.69 | loss: 0.60
epoch: 150 | accuracy: 0.69 | loss: 0.60

    ---
    epochs: 150
    total time: 299 seconds

    		 loss 	 train 	 test 
    init	 0.694 	 0.528 	 0.568
    end	 0.604 	 0.687 	 0.655
    ---


/home/lincoln/miniconda3/lib/python3.7/site-packages/torch/serialization.py:250: UserWarning: Couldn't retrieve source code for container of type CNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


''

In [22]:
figsize(15, 7)

plot_loss = plot(trainer._loss, label="loss")
plot_train = plot(trainer._accuracy_train, label="accuracy train")
plot_test = plot(trainer._accuracy_test, label="accuracy test")
plt.legend()
;